In [ ]:
!pip install transformers

In [2]:
import os
import pandas as pd
import shutil
import torch
import tensorflow as tf
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from torch.nn import functional as F
import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer, BertForMaskedLM
import random
from tqdm import tqdm

In [3]:
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [4]:
model_base = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict = True)
model_pre = BertForMaskedLM.from_pretrained("/content/drive/MyDrive/CS224N_Project/model_weights/bert_parler_6m/parler_6m", return_dict=True)
# Tell pytorch to run this model on the GPU.
#model_base.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
os.chdir("/content/drive/MyDrive/CS224N_Project")

# Prepare Sentence

In [6]:
#162 not in train
data = []
with open("/content/drive/MyDrive/CS224N_Project/data/Parler/parler_clean/parler_clean162.txt") as fp:
  for idx, line in enumerate(fp):
    if idx % 1000 == 0:
      data.append(line)


In [7]:
len(data)

527

In [16]:
text = data[9].strip()

In [17]:
text

'*ESSE É O CARA QUE OS SEGURANÇAS MATARAM*  Olha a ficha criminal do cara...  A ficha do anjinho, mártir e herói da esquerda. A violência aplicada foi exagerada e desnecessária, e os seguranças foram cruéis e despreparados, mas o sujeito era um violento costumaz, e sobre ele há muitas denúncias de porte de drogas e agressões, inclusive contra a esposa, em quem batia constantemente.   É preciso destacar que no incidente ocorrido, ele antes agrediu uma funcionária do Carrefour, e em seguida deu um soco no segurança. Esse é o "Santo" da patota lacradora!'

In [42]:
g = np.array(ground)
b = np.array(base_guess)
p = np.array(pre_guess)

In [44]:
np.savetxt("ground_truth_guess.txt", g,  fmt='%s')

In [45]:
np.savetxt("base_guess.txt", b,  fmt='%s')

In [46]:
np.savetxt("pre_guess.txt", p,  fmt='%s')

In [ ]:
ground = []
base_guess = []
pre_guess = []
base_right = 0
pre_right = 0
total = 0
for sen in tqdm(data):
  text_raw = sen.strip()
  word_sent = text_raw.split(" ")
  for mask_id in range(min(len(word_sent), 500)):
    mask_id = random.randint(0, len(word_sent) - 1)
    try:
      word_sent = text_raw.split(" ")
      gt = word_sent[mask_id]
      word_sent[mask_id] = tokenizer.mask_token
      text = ""
      for w in word_sent:
        text += w + " "
      #print(text)
      base = get_word(model_base, text)
      pre = get_word(model_pre, text)
      base_guess.append(base)
      pre_guess.append(pre)
      ground.append(gt)
      if base.lower() == gt.lower():
        base_right += 1
        print("BASE", base.lower(), gt.lower())
      if pre.lower() == gt.lower():
        print("PRE", pre.lower(), gt.lower())
        pre_right += 1
      total += 1
    except Exception as e:
      print(e)
      print(text_raw)
      print(word_sent)
      break

In [41]:
print(base_right, pre_right, total)

2231 1317 5706


Scores: 2231 1317 5706

Scores: 144 132 527

In [36]:
pre_guess

['great',
 'no',
 'you',
 'you',
 'president',
 '#',
 'you',
 'it',
 'welcome',
 'e',
 'eyes',
 'we',
 'senators',
 'you',
 '##ك',
 'bravo',
 'how',
 'this',
 'i',
 '##nies',
 '@',
 '##ano',
 'be',
 'the',
 'money',
 'is',
 'is',
 '##ل',
 'is',
 'you',
 '##son',
 'you',
 'of',
 'in',
 'for',
 'just',
 'prison',
 '##م',
 'yes',
 '##ا',
 'you',
 'oh',
 'you',
 'i',
 'care',
 'police',
 '##1',
 'doing',
 '@',
 'very',
 'you',
 'our',
 '##a',
 'this',
 'have',
 'not',
 'schools',
 'you',
 'and',
 'you',
 'cancel',
 'been',
 'be',
 'from',
 'no',
 'and',
 'life',
 'them',
 'own',
 'i',
 'crimes',
 'not',
 'praying',
 'flynn',
 'a',
 'test',
 'and',
 'for',
 'about',
 'is',
 'is',
 'out',
 'of',
 'i',
 'arrest',
 'de',
 'will',
 'video',
 'and',
 'in',
 'it',
 'about',
 'is',
 'be',
 'china',
 'a',
 'like',
 'you',
 'he',
 'stop',
 'praying',
 'down',
 'and',
 'good',
 'our',
 'portland',
 'you',
 'it',
 'is',
 'should',
 'yes',
 'it',
 'not',
 'you',
 'the',
 '@',
 'who',
 'what',
 'down',


In [37]:
base_guess

['partial',
 'public',
 '|',
 '|',
 ';',
 'crimes',
 '|',
 'america',
 'links',
 'a',
 'eyes',
 'you',
 'would',
 '|',
 '##ي',
 '[UNK]',
 'that',
 'that',
 '-',
 ';',
 '[UNK]',
 'trust',
 'take',
 'the',
 'sense',
 'too',
 'is',
 '-',
 '-',
 'you',
 '##s',
 'you',
 'of',
 'with',
 'and',
 'just',
 'school',
 '[UNK]',
 '-',
 '[UNK]',
 'i',
 '"',
 'they',
 ';',
 'feel',
 'police',
 '##e',
 'in',
 'u',
 'night',
 'like',
 'czech',
 ';',
 'it',
 '@',
 'now',
 'politics',
 ';',
 '##dam',
 '|',
 "'",
 'been',
 'be',
 '-',
 'a',
 ';',
 ';',
 'them',
 ';',
 'i',
 'votes',
 '"',
 ';',
 ';',
 'a',
 ';',
 'that',
 'of',
 '-',
 'cathedral',
 ';',
 'in',
 'of',
 'i',
 ';',
 'de',
 'will',
 ';',
 'on',
 'in',
 'tv',
 'about',
 'of',
 'be',
 'can',
 'a',
 'of',
 'you',
 'meat',
 'keep',
 'ready',
 'thing',
 'all',
 "'",
 'my',
 'he',
 'you',
 'it',
 ';',
 'i',
 'and',
 'it',
 'not',
 ';',
 'the',
 '##9',
 'who',
 'what',
 'over',
 '##y',
 'a',
 'supporting',
 'all',
 'website',
 'justice',
 '|',
 'ki

In [38]:
ground

['overflight',
 'Not',
 '#BestDayEver',
 '#GunsSaveLives',
 'favorite',
 '#WWG1WGA',
 '#ericfartwell',
 'it',
 'answers',
 'em',
 'nose',
 'Trump',
 'Republicans',
 ';utm_medium=parlerwj&#038;utm_campaign=parler&#038;utm_content=2020-06-24',
 'ما',
 '@AmericanGrandmaCrafts',
 'if',
 'That....',
 'not',
 'side!',
 '@CodyWrites',
 'follow',
 'know?',
 'righteousness',
 'sense',
 'here',
 'also!',
 '@Hamzaalenizy40',
 'Fortunately;',
 'others',
 'Stupid',
 'You',
 'of',
 'with',
 'after',
 'just',
 'college',
 'گورت',
 '@Nadeemalsmet1234',
 'والله',
 'I',
 '@Preachfreedom',
 'people',
 'then.',
 'feel',
 'BLM',
 '...but',
 'worth',
 '@ThatsThePointPB',
 'Saturday',
 'like...actual',
 '@WICPA77',
 '😂😂😂😂',
 'it',
 '😜',
 'not',
 'politics,',
 'dude',
 'given',
 '@waynedupreeshow',
 'cancel',
 'been',
 'handle',
 'HEADS',
 'a',
 'Kerry.',
 'contents.',
 'them',
 'a..',
 'I',
 'enablers',
 '🙏🇺🇸🇬🇧🇺🇸🇬🇧',
 'Amen!🙏✝️',
 'Flynn!⭐⭐⭐❤🇺🇸',
 'a',
 'also.',
 'saying',
 'for',
 'about',
 'royal',
 'reser

In [42]:
get_word(model_base, text)

'public'

In [44]:
get_word(model_pre, text)

'no'

In [14]:
#text_to_gen = text + " " + tokenizer.mask_token
def get_word(model, text):
  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  output = model(**input)
  logits = output.logits
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]
  top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
  #first = top_10[0]
  for token in top_10:
    word = tokenizer.decode([token])
    if word != "!" and word != "." and word != "," and word != "?" and word != ":" and word != ")":
      new_sentence = text.replace(tokenizer.mask_token, word)
      break
  return word
#   text = new_sentence
# print(new_sentence)